# import bilbliotek

In [1]:
import os
import json
import requests
from bs4 import BeautifulSoup

## Narzędzia


In [10]:
def extract(ancestor,selector= None, atribute = None, many = False):
    if selector:
        if many:
            if atribute:
                return [item[atribute].strip() for item in ancestor.select(selector)]
            return [item.text.strip() for item in ancestor.select(selector)]
        if atribute:
            try:
                return ancestor.select_one(selector)[atribute].strip()
            except TypeError: 
                return None
        try:

            return ancestor.select_one(selector).text.strip()
        except AttributeError:
            return None
    if atribute:
        try:
            return ancestor[atribute]
        except TypeError:
            return None
    return ancestor.text.strip()

In [13]:
opinion_schema = {
                    "review_id":(None,'data-entry-id'),
                    "author":(".user-post__author-name",),
                    "recomendation":(".user-post__author-recomendation > em",),
                    "stars":(".user-post__score-count",),
                    "content":(".user-post__text",),
                    "pros":("review-feature__item review-feature__item--positive",None,True),
                    "cons":("review-feature__item review-feature__item--negative",None,True),
                    "likes":(".vote-yes > span",),
                    "dislikes":(".vote-no > span",),
                    "publish_date":(".user-post__published > time:nth-child(1)",'datetime'),
                    "purchase_date":(".user-post__published > time:nth-child(2)",'datetime')

                }

## pobieranie opini o konkretnym produkcie

In [2]:
with open("headers.json","r") as file:
    headers = json.load(file)

In [14]:
product_id = "84514582"
next_page = f"https://www.ceneo.pl/{product_id}#tab=reviews"
all_opinions = []
while next_page:
    response = requests.get(next_page, headers=headers)
   #print(response.text)
    if response.status_code == 200:
        page_dom = BeautifulSoup(response.text, "html.parser")
        opinions = page_dom.select(".js_product-review:not(.user-post--highlight)")
        print(len(opinions))
        for opinion in opinions:
                single_opinion = {
                   key: extract(opinion,value)
                   for key , *value in opinion_schema.items()

                }

                all_opinions.append(single_opinion)

            
        try:
            next_page = "https://www.ceneo.pl"+ extract(page_dom,".pagination__next","href")
            
        except TypeError:
            next_page = None
print(all_opinions)

10


TypeError: unhashable type: 'list'

## Zapisanie wszystkich opini o konkretnym produkcie w bazie danych

In [4]:
if not os.path.exists("./opinions"):
    os.mkdir("./opinions")

In [5]:
with open(f"./opinions/{product_id}.json" , "w", encoding="utf-8") as jf:
    json.dump(all_opinions, jf, indent = 4, ensure_ascii=False)